In [ ]:
from plaid_bridges.dataloaders import PlaidDataLoader

from datasets import load_dataset
from plaid.containers.sample import Sample
import pickle
from plaid.bridges.huggingface_bridge import (
    huggingface_dataset_to_plaid,
    huggingface_description_to_problem_definition,
)

/home/fabien/miniconda3/envs/plaid-bridges/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Case with homogeneous samples

In [2]:
hf_dataset = load_dataset("PLAID-datasets/VKI-LS59", split="all_samples[:10]")
sample = Sample.model_validate(pickle.loads(hf_dataset[0]["sample"]))

pb_def = huggingface_description_to_problem_definition(hf_dataset.info.description)
ids = pb_def.get_split("train")[:10]


dataset, _ = huggingface_dataset_to_plaid(hf_dataset, ids=ids, processes_number=5)

Converting huggingface dataset to plaid dataset...


100%|██████████| 10/10 [00:00<00:00, 75.60it/s]


In [3]:
print(dataset)
all_feat_ids = dataset[0].get_all_features_identifiers()

scalar_features = [f for f in all_feat_ids if "scalar" in f.values()]
field_features = [f for f in all_feat_ids if "field" in f.values()]

in_feature_identifiers = [scalar_features[0], field_features[0]]
out_feature_identifiers = [field_features[1], scalar_features[1]]

print(in_feature_identifiers)
print(out_feature_identifiers)

Dataset(10 samples, 8 scalars, 0 time_series, 8 fields)
[{'type': 'scalar', 'name': np.str_('Pr')}, {'type': 'field', 'name': 'mach', 'base_name': 'Base_2_2', 'zone_name': 'Zone', 'location': 'Vertex', 'time': np.float64(0.0)}]
[{'type': 'field', 'name': 'rou', 'base_name': 'Base_2_2', 'zone_name': 'Zone', 'location': 'Vertex', 'time': np.float64(0.0)}, {'type': 'scalar', 'name': np.str_('Q')}]


In [4]:
loader = PlaidDataLoader(dataset, batch_size = 2, shuffle = True, in_feature_identifiers = in_feature_identifiers, out_feature_identifiers = out_feature_identifiers)

def _make_hashable(feat_dict: dict):
    return tuple(sorted(feat_dict.items()))

batch = next(iter(loader))
print("scalars =", batch[0][_make_hashable(in_feature_identifiers[0])])
print("fields =", batch[1][_make_hashable(out_feature_identifiers[0])], " | >>>> tensor:", batch[1][_make_hashable(out_feature_identifiers[0])].shape)

scalars = tensor([0.9525, 0.9636], dtype=torch.float64)
fields = tensor([[0.3300, 0.3301, 0.3301,  ..., 0.3353, 0.3352, 0.3351],
        [0.3269, 0.3269, 0.3270,  ..., 0.3414, 0.3414, 0.3413]],
       dtype=torch.float64)  | >>>> tensor: torch.Size([2, 36421])


## Case with heterogeneous samples

In [5]:
hf_dataset = load_dataset("PLAID-datasets/tensile2d", split="all_samples[:10]")
sample = Sample.model_validate(pickle.loads(hf_dataset[0]["sample"]))

pb_def = huggingface_description_to_problem_definition(hf_dataset.info.description)
ids = pb_def.get_split("train_500")[:10]


dataset, _ = huggingface_dataset_to_plaid(hf_dataset, ids=ids, processes_number=5)

Converting huggingface dataset to plaid dataset...


100%|██████████| 10/10 [00:00<00:00, 279.68it/s]


In [6]:
print(dataset)
all_feat_ids = dataset[0].get_all_features_identifiers()

scalar_features = [f for f in all_feat_ids if "scalar" in f.values()]
field_features = [f for f in all_feat_ids if "field" in f.values()]

in_feature_identifiers = [scalar_features[0], field_features[0]]
out_feature_identifiers = [field_features[1], scalar_features[1]]

print(in_feature_identifiers)
print(out_feature_identifiers)

Dataset(10 samples, 10 scalars, 0 time_series, 6 fields)
[{'type': 'scalar', 'name': np.str_('P')}, {'type': 'field', 'name': 'U1', 'base_name': 'Base_2_2', 'zone_name': 'Zone', 'location': 'Vertex', 'time': np.float64(0.0)}]
[{'type': 'field', 'name': 'sig22', 'base_name': 'Base_2_2', 'zone_name': 'Zone', 'location': 'Vertex', 'time': np.float64(0.0)}, {'type': 'scalar', 'name': np.str_('max_U2_top')}]


In [7]:
loader = PlaidDataLoader(dataset, batch_size = 2, shuffle = True, in_feature_identifiers = in_feature_identifiers, out_feature_identifiers = out_feature_identifiers)

def _make_hashable(feat_dict: dict):
    return tuple(sorted(feat_dict.items()))

batch = next(iter(loader))
print("scalars =", batch[0][_make_hashable(in_feature_identifiers[0])])
print("fields =", batch[1][_make_hashable(out_feature_identifiers[0])], " | >>>> list:", type(batch[1][_make_hashable(out_feature_identifiers[0])]))

scalars = tensor([-47.4500, -47.7900], dtype=torch.float64)
fields = [array([-1.65194049e-02, -1.83891654e-02, -1.58058256e-01, ...,
        4.62158966e+00, -1.49136043e+00,  3.45564270e+01], shape=(6263,)), array([-5.96606731e-03, -1.55568108e-01, -7.12685585e-02, ...,
        2.49899048e+02,  1.81157944e+02,  2.83986397e+01], shape=(6184,))]  | >>>> list: <class 'list'>
